In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install xla for tpu support
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [3]:
!pip install rasterio
#pip install pytorch-lightning
!pip install pytorch-lightning-bolts
#!pip install albumentations
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

     |████████████████████████████████| 18.1MB 1.2MB/s 
     |████████████████████████████████| 143kB 4.5MB/s 
     |████████████████████████████████| 481kB 8.9MB/s 
     |████████████████████████████████| 829kB 12.8MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 276kB 19.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=752a28180cb59bb8427cbe2c1d5a65ca55b7241f06bf261af29e073f3212a9c7
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=8bd8c64323897b82de88f9f63b595b4fc8d661b22084017905707ccf01b81e27
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built future PyYAML
  Found existing installation: future 0.16.0
    Uninstalling future-0.16.0:
      Successfully uninstalle

In [4]:
import numpy as np
import pandas as pd
import rasterio

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.metrics import MeanAbsoluteError
#import albumentations as A

#from sklearn.preprocessing import MinMaxScaler    
from sklearn.metrics import r2_score

import glob
import os

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
csv_path = '/content/drive/My Drive/omdena/hdi_with_geometry.csv'
root_dir = '/content/drive/My Drive/Landsat/'

In [6]:
class MyDataset(Dataset):
    """
    Generate normalized, rescaled and transformed datasets
    """

    def __init__(self, dataset_type, transform=None):
        
        super().__init__()
        self.df = dataset_type
        self.transform = transform

    def __len__(self):
        
        return len(self.df)
    
    def __getitem__(self, idx):
       
        if torch.is_tensor(idx):
              idx = idx.tolist()

        # generate image sample
        image_path = self.df['image_path'].iloc[idx] 
        image_sample = rasterio.open(str(image_path), "r")
        bands = [i for i in range(1, image_sample.count+1)]
        image_sample = image_sample.read(bands)
        image_sample = image_sample.astype('float32')

        # generate hdi sample

        hdi_sample = self.df['HDI'].iloc[idx]

        # Normalize the image sample and rescale it between 0 and 1
        for ch in range(image_sample.shape[0]):
            # standardize
            image_sample[ch] = (image_sample[ch] - np.nanmean(image_sample[ch])) / np.nanstd(image_sample[ch])
            
            # normalize
            image_sample[ch] = (image_sample[ch] - np.nanmin(image_sample[ch])) / (np.nanmax(image_sample[ch]) - np.nanmin(image_sample[ch]))
        
        # convet nan to 0
        image_sample[np.isnan(image_sample)] = 0

        if self.transform:
            image_sample = self.transform(image_sample)

        #print('hdi_sample', hdi_sample)

        return [image_sample.permute(1, 0, 2),  hdi_sample.astype('float32')]
    

In [7]:
model = torchvision.models.segmentation.fcn_resnet50() 
#model

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [8]:
model.backbone.conv1 = nn.Conv2d(6, 64, kernel_size=3, stride=1, padding=3, bias=False)
model.classifier[4] = nn.Sequential(nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1)), nn.Sigmoid())

In [9]:
from pytorch_lightning.callbacks import EarlyStopping

# default used by the Trainer
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=4,
    strict=False,
    verbose=True,
    mode='min')

from pytorch_lightning.callbacks import ModelCheckpoint

# DEFAULTS used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/My Drive/omdena/ckpt/model.ckpt',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    prefix=''
)


In [10]:
def my_collate(batch):

    max_wh = 0

    for item in batch:
        image = item[0]
        w = image.shape[1]
        h = image.shape[2]
        max_i = np.max([w, h])
        if max_i > max_wh:
            max_wh = max_i
    
    #print(max_wh)

    data = []

    for item in batch:
        image = item[0]
        rows = image.shape[1]
        cols = image.shape[2]
        rows_diff = max_wh - rows
        cols_diff = max_wh - cols
        cols_half = int(cols_diff / 2)
        rows_half = int(rows_diff / 2)
        padding = (cols_half, cols_diff-cols_half, rows_half, rows_diff-rows_half)
        image_pad = F.pad(image, padding, 'constant', 0)
        #print(image.shape)
        #print(image_pad.shape)
        #print(type(image_pad))
        data.append(image_pad)

    target = [item[1] for item in batch]
    return [data, target]

In [11]:
class Model(pl.LightningModule):

    def __init__(self, model, batch_size=1, learning_rate=.001):
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.model = model
        self.batch_size = batch_size

    def forward(self, x):
        x = self.model(x)
        output = torch.mean(x['out'], (1, 2, 3))
        return output

    def prepare_data(self):

        df = pd.read_csv(csv_path)
        df['image_path'] = root_dir + df['unique code'].astype(str) + '.tif' 
        df = df.sample(frac=1, random_state=1).reset_index(drop=True)
        # split the dataset
        train, validate, test = np.split(df, [int(.95*len(df)), int(.975*len(df))]) 

        # transforms
        train_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])

        validate_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])
        # create datasets for training, validation and test
        self.train_dataset = MyDataset(dataset_type=train, transform=train_transform)
        self.validate_dataset = MyDataset(dataset_type=validate, transform=validate_transform)
        self.test_dataset = MyDataset(dataset_type=test, transform=validate_transform) 
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=True, num_workers=4, collate_fn=my_collate, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.validate_dataset, self.batch_size, num_workers=4,collate_fn=my_collate, pin_memory=True) 

    def test_dataloader(self):
        return DataLoader(self.test_dataset, self.batch_size, num_workers=4, collate_fn=my_collate, pin_memory=True) 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.mse_loss(y_hat, y)
        self.log('train_loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        #print(y, y_hat)
        loss = F.mse_loss(y_hat, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_R-square', r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()), on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    #def validation_epoch_end(self, outputs):
    #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #  return {'val_loss': avg_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        y = torch.cuda.FloatTensor(y)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y)
        self.log('test_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_R-square', r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()),  on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def backward(self, trainer, loss, optimizer, optimizer_idx):
        loss.backward()

    def optimizer_step(self, current_epoch, batch_idx, optimizer, 
      optimizer_idx, second_order_closure=None, 
       on_tpu=False, using_native_amp=False, using_lbfgs=False):
        optimizer.step()


In [14]:
# init model
model_one = Model(model, batch_size=8)
#from pytorch_lightning.core.memory import ModelSummary
#ModelSummary(model_one, mode='full')

In [15]:
#seed
pl.seed_everything(1234)

from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger('/content/drive/My Drive/omdena/tb_logs', name='my_model')

#train
root_path = '/content/drive/My Drive/omdena'
trainer = pl.Trainer(gpus=1, precision=16, logger=logger, checkpoint_callback=checkpoint_callback, limit_test_batches=1, limit_train_batches=1,\
                    limit_val_batches=1, progress_bar_refresh_rate=50, accumulate_grad_batches=8, fast_dev_run=False,\
                    default_root_dir=root_path, auto_lr_find=True, gradient_clip_val=0.5,\
                    profiler=True,  max_epochs=1000, callbacks=[early_stop, PrintTableMetricsCallback()])

trainer.fit(model_one)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.

  | Name  | Type | Params
-------------------------------
0 | model | FCN  | 32 M  


RuntimeError: ignored

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir='/content/drive/My Drive/omdena/tb_logs'

In [ ]:
# test
trainer.test()

In [ ]:
from sklearn.metrics import mean_squared_error
A = [0.4062, 0.2288, 0.4386, 0.4160]
B = [0.4140, 0.3354, 0.6566, 0.6081]

mse = mean_squared_error(A, B)
mse